In [1]:
import schedule_Generator as sg
import pandas as pd

Example
- Line 25: 25
- Stop ULB: 5407F
- Direction BOONDAEL GARE
- Date: SEP 07

In [34]:
trips = pd.read_csv('../data/gtfs3Sept/trips.csv')
trips_line = trips.loc[trips['route_id']==25,:]
trip_line_head = trips_line.loc[trips_line['trip_headsign']=='BOONDAEL GARE',:]
trip_line_head.head()

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id
96068,25,236896051,113149920236896051,BOONDAEL GARE,1,8925469,025t0256
96069,25,236896051,113149922236896051,BOONDAEL GARE,1,8925474,025t0256
96072,25,236896051,113149925236896051,BOONDAEL GARE,1,8925469,025t0256
96075,25,236896051,113149929236896051,BOONDAEL GARE,1,8925473,025t0256
96077,25,236896051,113149932236896051,BOONDAEL GARE,1,8925474,025t0256


In [37]:
calendar = pd.read_csv('../data/gtfs3Sept/calendar.csv')
calendar_week = calendar.loc[calendar['start_date']==20210906,:]
calendar_week_day = calendar_week.loc[calendar_week['tuesday']==1,:]
calendar_week_day.head()

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
15,237645001,1,1,1,1,1,0,0,20210906,20210917
26,237648001,1,1,1,1,1,0,0,20210906,20210917
100,238202001,1,1,1,1,1,0,0,20210906,20210917
203,238119001,1,1,1,1,1,0,0,20210906,20210917
223,238201001,1,1,1,1,1,0,0,20210906,20210917


In [38]:
trip_line_date_head = pd.merge(left=trip_line_head, right=calendar_week_day, on='service_id', how='inner').loc[:,['route_id','service_id','trip_id']]
trip_line_date_head.head()

,route_id,service_id,trip_id
0,25,238202001,113480343238202001
1,25,238202001,113480346238202001
2,25,238202001,113480348238202001
3,25,238202001,113480356238202001
4,25,238202001,113480357238202001


In [77]:
stop_times = pd.read_csv('../data/gtfs3Sept/stop_times.csv')
time_line_date_head = pd.merge(left=trip_line_date_head, right=stop_times, on='trip_id')
time_line_date_head_stop = time_line_date_head.loc[time_line_date_head['stop_id']=="5407F",:]
time_line_date_head_stop.sort_values('arrival_time').count()

route_id          146
service_id        146
trip_id           146
arrival_time      146
departure_time    146
stop_id           146
stop_sequence     146
pickup_type       146
drop_off_type     146
dtype: int64

In [41]:
def unix_to_datetime(unix_time):
    return pd.to_datetime(unix_time,unit="ms",origin="unix")

def datetime_to_unix(dt):
    return int(pd.Timestamp.timestamp(pd.Timestamp(dt))*1000)

unix_to_datetime(1630998894287)

Timestamp('2021-09-07 07:14:54.287000')

In [83]:
actural_time = pd.concat([pd.read_csv('../data/vehiclePosition01.csv'),pd.read_csv('../data/vehiclePosition02.csv'),pd.read_csv('../data/vehiclePosition03.csv')])
actural_time_line = actural_time.loc[actural_time['LineID']==25,:]
actural_time_line_point = actural_time_line.loc[actural_time_line['PointID']==5407,:]
actural_time_line_point['Time'] = actural_time_line_point['Time'].apply(unix_to_datetime)
actural_time_line_point_date = actural_time_line_point.loc[actural_time_line_point['Time'].dt.date == pd.to_datetime('2021-09-07').date(),:]
actural_time_line_point_date_arrive = actural_time_line_point_date.loc[actural_time_line_point_date['DistanceFromPoint']==0,:]
select_list = [True]
for i in range(1, len(actural_time_line_point_date_arrive)):
    if actural_time_line_point_date_arrive.iloc[i,0] - actural_time_line_point_date_arrive.iloc[i-1,0] <= pd.Timedelta('00:00:45'):
        select_list.append(False)
    else:
        select_list.append(True)
actural_time_line_point_date_arrive_noduplicate = actural_time_line_point_date_arrive.loc[select_list,:]
actural_time_line_point_date_arrive_noduplicate.count()

/var/folders/v1/m2nqhx3d4yv9mjbpndrlsck00000gn/T/ipykernel_13291/2597911513.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actural_time_line_point['Time'] = actural_time_line_point['Time'].apply(unix_to_datetime)


Time                 62
LineID               62
DirectionID          62
DistanceFromPoint    62
PointID              62
dtype: int64

Schedule train 146
Actual train 62

In [134]:
on_time = 0
time_line_date_head_stop['arrival_time'].apply(pd.to_timedelta)
actural_time_line_point_date_arrive_noduplicate['Time'] - pd.Timestamp('2021-09-07 00:00:00')
for t in time_line_date_head_stop['arrival_time'].apply(pd.to_timedelta):
    ot = False
    for at in actural_time_line_point_date_arrive_noduplicate['Time'] - pd.Timestamp('2021-09-07 00:00:00'):
        t_dif = t - at
        if t_dif <= pd.Timedelta('00:00:30') and t_dif >= pd.Timedelta('-00:00:30'):
            ot = True
    if ot == True:
        on_time+=1
on_time_rate = on_time/len(time_line_date_head_stop['arrival_time'])
on_time_rate

0.0821917808219178